In [5]:
import os
import pandas as pd
import re
import numpy as np

In [6]:
pd.set_option('display.width', 5000)
pd.set_option('max_colwidth', -1)

/var/folders/rv/22bxnp5j4mb8zqzkrj7vz_hc0000gn/T/ipykernel_31734/1620929901.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [7]:
lang = "en"
data = "test"
path = f"../{lang}/{data}"

for f in os.listdir(path):
    n = re.search("(?<=large-).+(?=\.csv)", f)
    if n:
        if n.group(0) == "LM":
            LM_df = pd.read_csv(os.path.join(path, f))
        elif n.group(0) == "tags":
            tags_df = pd.read_csv(os.path.join(path,f))
    else:
        none_df = pd.read_csv(os.path.join(path, f))

In [8]:
LM_df['predictions'].shape

(217818,)

In [9]:
none_df['predictions'].shape

(217818,)

In [10]:
all_df = none_df
all_df['LM_pred'] = LM_df['predictions']
all_df['tags_pred'] = tags_df['predictions']
all_df = all_df.rename(columns={"predictions": "none_pred"})
all_df

,ID,sent,none_pred,true,LM_pred,tags_pred
0,724bdabd-251d-4f6d-a939-cf96dc07b4fa,ford cargo price,B-PROD I-PROD O,B-PROD I-PROD O,B-PROD I-PROD O,B-PROD I-PROD O
1,f19c0592-10fd-447d-8874-4248b956a678,helcom wikipedia,B-LOC O,B-GRP O,B-PER B-CW,B-CORP O
2,632fae32-405a-4d3e-90b6-8ca6067841b2,"what is sparta, michigan",O O B-LOC B-LOC,O O B-LOC I-LOC,O O B-LOC I-LOC,O O B-LOC I-LOC
3,715e624c-fc85-42e6-8192-71bf52a88909,"barwich died in cologne , aged 54 .",O O O B-LOC O O O O,O O O B-LOC O O O O,O O O B-LOC O O O O,O O O B-LOC O O O O
4,40a23637-5e3c-41f5-aeb4-1917b97655b4,how many calories in a svoronos,O O O O O B-PROD,O O O O O B-LOC,O O O O O B-PROD,O O O O O B-LOC
...,...,...,...,...,...,...
217813,306a910d-9fa7-4752-a937-b20b6e4da622,play play on free online,O B-CW I-CW I-CW O,O B-CW I-CW O O,O B-CW I-CW I-CW O,O B-CW I-CW I-CW O
217814,ca56fdff-ad66-4094-abe3-b8f7a12a8d0c,lake naivasha crime news,B-LOC I-LOC O O,B-LOC I-LOC O O,B-LOC I-LOC O O,B-LOC I-LOC O O
217815,5a21b3ab-ee6a-46e0-827e-66b66c9f2f7e,rademacher grew up in decatur .,O O O O B-LOC O,O O O O B-LOC O,O O O O B-LOC O,O O O O B-LOC O
217816,2eea6675-c0b4-425e-89a0-3f9b82e62531,a political pensioner enjoys a pension awarded due to his or political career or significance .,O O O O O B-PROD O O O O O O O O O O,O O O O O B-PROD O O O O O O O O O O,O O O O O O O O O O O O O O O O,O O O O O B-PROD O O O O O O O O O O


In [30]:
none_lm_diff = np.where((all_df['none_pred'] != all_df['LM_pred']) & (all_df['true'] == all_df['LM_pred']))
print("output diff between none and LM: ", len(none_lm_diff[0]))
none_lm_df = all_df.iloc[none_lm_diff]
none_lm_df = none_lm_df.rename(columns = {"sent":"none_sent"})
none_lm_df['lm_sent'] = LM_df['sent']

none_lm_sent_diff = np.where(none_lm_df['none_sent'] != none_lm_df['lm_sent'])
print("sent diff + output diff between none and LM: ", len(none_lm_sent_diff[0]))
none_lm_df[["ID", "none_sent", "lm_sent", "none_pred", "LM_pred", "true"]].iloc[none_lm_sent_diff]
#none_lm_df.iloc[none_lm_sent_diff]
none_lm_sent_df = none_lm_df[["ID", "none_sent", "lm_sent", "none_pred", "LM_pred", "true"]].iloc[none_lm_sent_diff]
none_lm_sent_df = none_lm_sent_df.sort_values(by="lm_sent", key=lambda x: x.str.len()).head(30)
none_lm_sent_df

output diff between none and LM:  12020
sent diff + output diff between none and LM:  5026


,ID,none_sent,lm_sent,none_pred,LM_pred,true
118016,113bb311-9649-43c9-a209-f7185bbe01f7,wbc wa,wbc wa wbc may stand for:,B-CORP I-CORP,B-CORP O,B-CORP O
172187,339d7773-f404-483a-a930-270ece539652,kuki com login,kuki com login kuki can refer to:,B-CW I-CW O,B-CORP O O,B-CORP O O
41981,770bca4e-e8fa-4ccd-aeca-41f3dc6a5388,yahoo ratings,"yahoo ratings yahoo! (, styled yahoo!",B-CORP I-CORP,B-CORP O,B-CORP O
42030,6e770334-0e46-4e7f-b968-52b385cfae94,nh us route 65,nh us route 65 nh or nh may refer to,O O I-LOC I-LOC,O B-LOC I-LOC I-LOC,O B-LOC I-LOC I-LOC
192262,cb96428b-06c6-4ad2-8674-7a5ec974ac9e,yahoo borgo fc,"yahoo borgo fc yahoo! (, styled yahoo!",B-CORP B-GRP I-GRP,O B-GRP I-GRP,O B-GRP I-GRP
63669,b785dbb1-5986-417e-b13e-fb3e0806ac87,k m peyton movie,"k m peyton movie km, km, or km may stand for:",B-PER O I-PER O,B-PER I-PER I-PER O,B-PER I-PER I-PER O
189748,309e53bf-d325-498b-80d8-5acfa12a97d8,willème 800,willème 800 willème was a french truck manufacturer.,B-PROD O,B-CORP O,B-CORP O
146945,11ace71f-8549-4be2-8224-c8bc04673e39,tree tony sandler,tree tony sandler sandler is a yiddish family name.,B-CW B-PER I-PER,O B-PER I-PER,O B-PER I-PER
142465,da13d359-008c-4058-b5c9-04118dd306ec,when is thomas blayney season,when is thomas blayney season blayney could refer to:,O O B-CW I-CW O,O O B-PER I-PER O,O O B-PER I-PER O
133116,5764bb87-0f03-4da4-a910-e6b482401575,define kotaro yanagi,define kotaro yanagi kotaro yanagi is a japanese actor.,O B-CW I-CW,O B-PER I-PER,O B-PER I-PER


In [12]:
none_tags_diff = np.where((all_df['none_pred'] != all_df['tags_pred']) & (all_df['true'] == all_df['tags_pred']))
print("output diff between none and tags: ", len(none_tags_diff[0]))
all_df.iloc[none_tags_diff]
none_tags_df = all_df.iloc[none_tags_diff]
none_tags_df = none_tags_df.rename(columns = {"sent":"none_sent"})
none_tags_df['tags_sent'] = tags_df['sent']

none_tags_sent_diff = np.where(none_tags_df['none_sent'] != none_tags_df['tags_sent'])
print("sent diff + output diff between none and tags: ", len(none_tags_sent_diff[0]))
none_tags_sent_df = none_tags_df[["ID", "none_sent", "tags_sent", "none_pred", "tags_pred", "true"]].iloc[none_tags_sent_diff]
none_tags_sent_df.sort_values(by="tags_sent", key=lambda x: x.str.len()).head(30)



output diff between none and tags:  42947
sent diff + output diff between none and tags:  38063


,ID,none_sent,tags_sent,none_pred,tags_pred,true
86293,99770bb0-0556-4c1a-b777-467b5c62ca2c,spr seedlings,spr seedlings spr can refer to:,O O,B-PROD O,B-PROD O
91297,9b5cb51f-4174-47de-ac26-53eb7443e9f8,backyard m-25,backyard m-25 m25 or m-25 may be:,B-PROD I-PROD,O B-PROD,O B-PROD
170296,11571b33-b83f-4b31-8bed-352a0f707e9c,pronar erie pa,pronar erie pa pronar sp. z o.o.,O B-PER I-PER,B-CORP O O,B-CORP O O
87072,15e5e970-f760-4a72-a274-60568d1cd10e,obsessed author,obsessed author obsessed may refer to,O O,B-CW O,B-CW O
173299,87c2c8f4-1c9c-425d-803f-c91686630c5c,mayor of arugba,mayor of arugba arugba is a 2008 film.,O O B-LOC,O O B-CW,O O B-CW
133121,58f64f7b-a502-4431-9c33-a909c3af6f04,movement for! mississippi,movement for! mississippi movement for!,B-GRP I-GRP B-LOC,B-GRP I-GRP O,B-GRP I-GRP O
216846,a28742a3-e58a-4ddc-b8dc-b23791190e56,define toikka,define toikka toikka is a finnish surname.,O B-CORP,O B-PER,O B-PER
96667,1f04c7eb-8e96-445a-a0da-5582f58b8a1c,where was nu pogodi,"where was nu pogodi well, just you wait!",O O B-LOC I-LOC,O O B-CW I-CW,O O B-CW I-CW
92160,d42fb906-0847-4f65-9ae4-6aca695c1c4a,wish ko lang wasp,wish ko lang wasp wish ko lang! (transl.,O O O B-PROD,B-CW I-CW I-CW O,B-CW I-CW I-CW O
154576,8f1f9b61-7f67-4b44-8517-b3ccb284169a,words to describe pto,words to describe pto pto may stand for:,O O O B-LOC,O O O B-GRP,O O O B-GRP


In [29]:
# where preds are different in all three methods
none_lm_tags_diff = np.where(((all_df['none_pred'] != all_df['tags_pred']) & (all_df['tags_pred'] != all_df['LM_pred']) & (all_df['none_pred'] != all_df['LM_pred'])) & ((all_df['true'] == all_df['tags_pred']) | (all_df['true'] == all_df['LM_pred']) | (all_df['true'] == all_df['none_pred'])))
print("num of output diff between all three: ", len(none_lm_tags_diff[0]))
none_lm_tags_df = all_df[["ID", "sent", "none_pred", "LM_pred", "tags_pred", "true"]].iloc[none_lm_tags_diff]
none_lm_tags_df['lm_sent'] = LM_df['sent']
none_lm_tags_df['tags_sent'] = tags_df['sent']
none_lm_tags_df = none_lm_tags_df.rename(columns = {"sent":"none_sent"})

#where the preds are different AND the sents are different 
none_others_sent_diff = np.where((none_lm_tags_df['none_sent'] != none_lm_tags_df['tags_sent']) | (none_lm_tags_df['none_sent'] != none_lm_tags_df['lm_sent']))

#none_lm_tags_sent_diff = set(none_tags_sent_diff[0]).intersection(none_lm_sent_diff[0])
#print("num of sent + output diff between all three:", len(none_lm_tags_sent_diff))
none_lm_tags_sent_df = none_lm_tags_df.iloc[none_others_sent_diff]

#none_lm_tags_sent_df[["ID", "none_sent", "lm_sent", "tags_sent", "none_pred", "LM_pred", "tags_pred", "true"]]
none_lm_tags_sent_df = none_lm_tags_sent_df.sort_values(by="true", key=lambda x: x.str.len()).head(30)
none_lm_tags_sent_df = none_lm_tags_sent_df[["ID", "none_sent", "lm_sent", "tags_sent", "none_pred", "LM_pred", "tags_pred", "true"]]
none_lm_tags_sent_df

num of output diff between all three:  17953


,ID,none_sent,lm_sent,tags_sent,none_pred,LM_pred,tags_pred,true
157917,97c98feb-806e-4e37-b4c7-f6d9eef2ea6a,tii help,tii help,"tii help tii may be,tii language of indonesiatiene language of drcongo tii , an estonian songtii, iata airport code for tarinkot airport, in southeastern afghanistantiye (c. 1398 bc – 1338 bc, wife of egyptian pharaoh amenhotep iii. transport infrastructure ireland",B-GRP O,B-CW O,O O,B-CW O
39204,3c573215-ab56-4486-954b-b12b07bdf14d,worms wikia,worms wikia fandom is a wiki hosting service that hosts wikis mainly on entertainment topics .,worms wikia,B-CW O,O O,B-CW I-CW,B-CW O
74219,57bd9938-c852-4110-a8df-7039cebef879,mindenki wikipedia,"mindenki wikipedia wikipedia is a multilingual free online encyclopedia written and maintained by a community of volunteers, known as wikipedians, through open collaboration and using a wiki-based editing system. wikipedia is the largest and most-read reference work in history.",mindenki wikipedia mindenki is the seventh hungarian language studio album by hungarian rock band locomotiv gt released in late autumn 1978. the band recorded the album without guest musicians.,O O,B-CW B-CW,B-CW O,B-CW O
160729,20b4fa03-06fe-4e0e-908f-2a8678fdbe50,memorial mammo,memorial mammo,"memorial mammo mammo is a 1994 indian hindi film by shyam benegal. it stars farida jalal, surekha sikri, amit phalke and rajit kapur.",O B-LOC,O I-CW,O B-CW,O B-CW
211732,f0e8d440-816b-4fd4-b840-7e854a7f26d9,reddit geist,"reddit geist reddit is an american social news aggregation, content rating, and discussion website. registered users submit content to the site such as links, text posts, images, and videos, which are then voted up or down by other members.","reddit geist geist is a german noun with a significant degree of importance in german philosophy. its semantic field corresponds to english ghost, spirit, mind, intellect.",B-CW B-CW,B-CORP O,O B-CW,O B-CW
191333,164d36e3-d45a-4cfb-b9e2-5515f4a4ffb9,rochester dogpile,rochester dogpile,"rochester dogpile dogpile is a metasearch engine for information on the world wide web that fetches results from google, yahoo!, yandex, bing, and other popular search engines, including those from audio and video content providers such as yahoo!.",B-GRP I-GRP,B-LOC I-LOC,O B-CW,O B-CW
106189,3a74361b-8983-44c6-a212-bc6f89b6321f,stream itchio,stream itchio,"stream itchio itch.io is a website for users to host, sell and download indie games. launched in march 2013 by leaf corcoran, the service hosts over 500,000 games and items as of april 2022.",O B-CW,O B-LOC,O B-CORP,O B-CW
107392,a9a31322-a69c-4ad7-8aee-460d462e4ca2,metro giftpia,metro giftpia,"metro giftpia giftpia , stylized as giftpia, is a video game, developed by skip ltd. for the gamecube. it was released in japan on april 25, 2003.",B-LOC I-LOC,O B-LOC,O B-CW,O B-CW
211795,5e17741c-1908-4407-a345-34e17d4505f2,hobo history,"hobo history a hobo is a migrant worker in the united states. hoboes, tramps and bums are generally regarded as related, but distinct: a hobo travels and is willing to work; a tramp travels, but avoids work if possible; and a bum neither travels nor works.","hobo history a hobo is a migrant worker in the united states. hoboes, tramps and bums are generally regarded as related, but distinct: a hobo travels and is willing to work; a tramp travels, but avoids work if possible; and a bum neither travels nor works.",B-CW O,O O,B-PER O,B-CW O
212024,3552acc7-fbe6-4c25-9178-2280185df613,özsoy wiki,özsoy wiki özsoy is an opera composed by ahmet adnan saygun from a libretto by münir hayri egeli. it was the first turkish opera composed during the country's republican period under mustafa kemal atatürk and it was given its premiere performance in the halk evi theatre in ankara on 19 june 1934.the opera tells the story of twin brothers tur and īraj in three parts.,özsoy wiki özsoy is an opera composed by ahmet adnan saygun from a libretto by münir hayri egeli. it was the first turkish opera compo

In [14]:
none_lm_sent_df.to_csv("none_lm_sent_sent_diff.csv")
none_tags_sent_df.to_csv("none_tags_sent_diff.csv")
none_lm_tags_sent_df.to_csv("none_tags_sent_diff.csv")